In [2]:
%pip install pandas snowflake-connector-python pyarrow

  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached snowflake_connector_python-4.1.0-cp313-cp313-win_amd64.whl.metadata (79 kB)
  Using cached numpy-2.3.5-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached pyopenssl-25.3.0-py3-none-any.whl.metadata (17 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached cffi-2.0.0-cp313-cp313-win_amd64.whl.metadata (2.6 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl (11.0 MB)
Using cached snowfl

In [16]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# 1. EXTRACT
# Replace with the path to your downloaded CSV file
csv_path = 'Warehouse_and_Retail_Sales.csv'
print("Reading CSV...")

df = pd.read_csv(csv_path)
df.info()



Reading CSV...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307645 entries, 0 to 307644
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   YEAR              307645 non-null  int64  
 1   MONTH             307645 non-null  int64  
 2   SUPPLIER          307478 non-null  object 
 3   ITEM CODE         307645 non-null  object 
 4   ITEM DESCRIPTION  307645 non-null  object 
 5   ITEM TYPE         307644 non-null  object 
 6   RETAIL SALES      307642 non-null  float64
 7   RETAIL TRANSFERS  307645 non-null  float64
 8   WAREHOUSE SALES   307645 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 21.1+ MB


In [ ]:
# 2. TRANSFORM
# Clean column names (Snowflake prefers uppercase or simple names)
df.columns = [c.upper().replace(' ', '_') for c in df.columns]


np.int64(2017)

In [21]:
#  checking how old the data is
df['YEAR'].min()
df['YEAR'].max()

np.int64(2020)

In [22]:
df = df[df['YEAR'] >=2019]
print(f"Data ready: {len(df)} rows")

Data ready: 184916 rows


In [30]:
# 3. LOAD to Snowflake
# Update these details with your Snowflake account info
conn = snowflake.connector.connect(
    user='LEMINE',
    password='#zuIhs$3KD@pyP@M2XU&',
    account='XORHYJX-DMC95475',
    warehouse='COMPUTE_WH',
    database='SECOND_DB',
    schema='SECOND_SCHEMA'
)
create_tbl_query = """
CREATE TABLE IF NOT EXISTS RETAIL_SALES (
    YEAR INT, 
    MONTH INT, 
    SUPPLIER VARCHAR, 
    ITEM_CODE VARCHAR, 
    ITEM_DESCRIPTION VARCHAR, 
    ITEM_TYPE VARCHAR, 
    RETAIL_SALES INT, 
    RETAIL_TRANSFERS INT, 
    WAREHOUSE_SALES INT
)
"""
conn.cursor().execute(create_tbl_query)

# Write data using the optimized write_pandas method
success, n_chunks, n_rows, _ = write_pandas(conn, df, 'RETAIL_SALES')
print(f"Success! Inserted {n_rows} rows.")
conn.close()

C:\Users\missa\AppData\Local\Temp\ipykernel_21884\3251323408.py:27: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, n_chunks, n_rows, _ = write_pandas(conn, df, 'RETAIL_SALES')


Success! Inserted 184916 rows.
